On met en place un modèle d'apprentissage supervisé pour classifier les blochets en (non)-exploitables. Une classification "à la main" a déjà été effectuée, et montre que la proportion de blochets non exploitables se situe autour de 18%: il s'agit donc d'un problème de classification non-équilibrée (imbalanced classification).

In [38]:
## IMPORTS
import math

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import copy
import tensorflow as tf

# Preprocessing
from sklearn.preprocessing import MinMaxScaler

import numpy.random as rd

import seaborn as sns

from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix

from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import callbacks
from tensorflow.keras import optimizers
from tensorflow.keras import metrics
from tensorflow.keras.initializers import VarianceScaling
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from keras.engine.topology import Layer, InputSpec
# from keras.optimizers import SGD

from sklearn import preprocessing
# from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split

import imblearn
# import metrics
import os

from time import time

On reprend la même procédure d'import des données que pour la classification à l'aide de la SOM.

In [2]:
#fonctions et variables utiles

def read_text_file(file_path):
    with open(file_path, encoding='ISO-8859-1') as f:
        lines = f.readlines()
        return lines
        
def read_number_line(some_line):
    return [float(s) for s in some_line.split()]

# my_dpi = np.sqrt(1920**2 + 1080**2)/17.3 #dpi de l'écran
# tnorm = 60. #normalisation du temps
# vnorm = 1.05 #normalisation de la vitesse

In [3]:
##dossiers où récupérer les données

#training
folder = "C:/Users/victo/stage_navier_jupyter/data_eurotunnel/Data/training/"
pk_folder = "1775_20190128/" #le PK qu'on va traiter

done = ["1775_20190128/", "1780_20190128/", "1785_20190128/", "1795_20190128/", "1800_20190128/", "1805_20190128/",
        "1810_20190128/", "1815_20190128/", "1820_20190128/", "1825_20190128/", "1830_20190128/", "1835_20190128/",
        "1840_20190128/", "3020_20190218/", "3025_20190218/", "3030_20190218/", "3035_20190218/", "3040_20190218/",
        "3045_20190218/", "3050_20190218/", "3055_20190218/", "3060_20190218/", "3065_20190218/", "3070_20190218/",
        "3075_20190218/", "3080_20190218/", "4230_20190225/", "4235_20190225/", "4240_20190225/", "4255_20190225/",
        "4260_20190225/"]
failed = ["1790_20190128/", "3015_20190218"]

# #validation
# folder = "C:/Users/victo/stage_navier_jupyter/data_eurotunnel/Data/validation/"
# pk_folder = "4250_20190225/" #le PK qu'on va traiter

done_val = ["4245_20190225/", "4250_20190225/"]

os.chdir(folder)



In [4]:
# path = folder + pk_folder #pour accéder aux fichiers .blochet du PK pk_folder

mySeries = [] #series pandas
namesofMySeries = [] #nom correspondant a chaque blochet

# os.chdir(path) #current directory: C:/Users/victo/stage_navier_jupyter/data_eurotunnel/Data/training/1775-20190128/

def read_text_file(file_path):
    with open(file_path, encoding='ISO-8859-1') as f:
        lines = f.readlines()
        return lines
    
y_true = []

for pk_folder_loop in os.listdir(folder):
    if (pk_folder_loop + "/") in done:
        path = folder + pk_folder_loop
        for file in os.listdir(path):
            if file.endswith(".blochet"):
                file_path = f"{path}/{file}"
                lines = read_text_file(file_path)
                
                y_true.append(int(lines[-1][-1]))

                times = []
                speeds = []
                freqs = []
                frfs_real = []
                frfs_imag = []

                lines_to_read = lines[15:len(lines)-1] #valeurs commencent ligne 16 et saut de ligne à la fin
                
                for i in range(len(lines_to_read)):
                    read_line = read_number_line(lines_to_read[i])
                    times.append(read_line[0])
                    speeds.append(read_line[1])
                    freqs.append(read_line[2])
                    frfs_real.append(read_line[3])
                    frfs_imag.append(read_line[4])

                dict = {"Temps":times, "Vitesses":speeds, "Fréquences":freqs, "FRF réel":frfs_real, "FRF imag":frfs_imag}
                df = pd.DataFrame(dict)

                df = df.loc[:, ["Temps", "Vitesses"]]
                df.set_index("Temps", inplace=True)

                mySeries.append(df)
                namesofMySeries.append(pk_folder_loop + "_" + file[:-8])

In [5]:
dict_series = {namesofMySeries[i]: mySeries[i] for i in range(len(mySeries))}
# print(dict_series.keys())
# print(dict_series.values())

In [6]:
len(mySeries)

4850

In [7]:
mySeries[42]

,Vitesses
Temps,
0.000000,0.001956
0.097656,0.002443
0.195312,0.001975
0.292969,0.003927
0.390625,0.002536
...,...
59.472660,-0.000771
59.570310,0.000244
59.667970,0.000449


In [8]:
y_true_values = {x for x in y_true}
print("Valeurs inscrites: " + str(y_true_values) + "\n" + 
      "Bonne longueur de liste: " + str(len(mySeries)==len(y_true)) + "\n" +
      "Pourcentage de blochets inexploitables: " + str(round(100*(1 - np.sum(y_true)/len(y_true)), 3)) + "%" + "\n")

#valeurs en float64

Valeurs inscrites: {0, 1}
Bonne longueur de liste: True
Pourcentage de blochets inexploitables: 18.186%



In [9]:
series_lengths = {len(series) for series in mySeries}
print(series_lengths)

{614}


In [12]:
for i in range(len(mySeries)):
    scaler = MinMaxScaler()
    mySeries[i] = MinMaxScaler().fit_transform(mySeries[i])
    mySeries[i]= mySeries[i].reshape(len(mySeries[i]))

In [13]:
print("max: "+str(max(mySeries[0]))+"\tmin: "+str(min(mySeries[0])))
print(mySeries[0][-5:])

max: 1.0	min: 0.0
[0.40812886 0.40739371 0.40760733 0.40815812 0.40736352]


On vectorise les données obtenues ($x$ représente les échantillons de chaque blochet et $y$ les valeurs d'exploitabilité) pour pouvoir les passer au modèle.

In [32]:
x = np.array(mySeries, dtype="float32")
print(x.shape)
y = np.array(y_true, dtype="uint8")
y = np.reshape(y, (y.shape[0], 1))
print(y.shape)

(4850, 614)
(4850, 1)


In [33]:
print(x[42])

[0.43668875 0.4370555  0.43670303 0.43817562 0.437126   0.4376144
 0.43649477 0.4357442  0.43716148 0.43652102 0.43789363 0.43794847
 0.4391713  0.43937728 0.44004813 0.43763283 0.44081575 0.44053468
 0.43544745 0.4379443  0.43340215 0.43301603 0.42847112 0.43067583
 0.427721   0.42901942 0.42784354 0.42715794 0.42858723 0.42816055
 0.4274519  0.42756388 0.42805827 0.42524487 0.4276491  0.4264023
 0.42641243 0.42678335 0.4290839  0.42943916 0.4304533  0.43138126
 0.43217975 0.4310956  0.43193695 0.43094677 0.4307841  0.42897335
 0.43063483 0.42795458 0.43136975 0.4291945  0.43331736 0.4311168
 0.43790606 0.43150198 0.44112584 0.42963222 0.4436969  0.42015857
 0.4679904  0.6194971  0.39961517 0.41887534 0.40219128 0.4148801
 0.4107084  0.4205668  0.41834223 0.42405012 0.4219394  0.42503154
 0.42237988 0.42453852 0.424398   0.42530525 0.42664006 0.4299704
 0.4297875  0.43255666 0.43351457 0.43562576 0.43587273 0.43740708
 0.43775493 0.43788072 0.43796825 0.4365975  0.43639848 0.43715042


On sépare ensuite les 4850 données en un ensemble d'entraînement et un ensemble de test selon une proportion 80/20 à l'aide d'une fonction pré-implémentée de scikit-learn. Le paramètre stratify = y indique que l'on veut une répartition la plus homogène possible des blochets inexploitables parmi les blochets exploitables, ce qui est essentiel dans un problème de classification non équilibrée comme celui-ci.

In [34]:
x_train_to_split, x_test, y_train_to_split, y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)

L'ensemble d'entraînement est ensuite lui-même séparé en un ensemble de training pur et un ensemble de validation, là aussi selon un principe 80/20.

In [35]:
num_val_samples = int(len(x_train_to_split) * 0.2)
x_train = x_train_to_split[:-num_val_samples]
y_train = y_train_to_split[:-num_val_samples]
x_val = x_train_to_split[-num_val_samples:]
y_val = y_train_to_split[-num_val_samples:]

print("Number of training samples:", len(x_train))
print("Number of validation samples:", len(x_val))


Number of training samples: 3104
Number of validation samples: 776


On analyse ensuite la répartition des classes dans l'ensemble de training pour affecter des poids différents aux deux classes (la classe "exploitable" codée par un "1" aura un poids plus faible).

In [36]:
counts = np.bincount(y_train[:, 0])
print(
    "Number of positive samples in training data: {} ({:.2f}% of total)".format(
        counts[1], 100 * float(counts[1]) / len(y_train)
    )
)

weight_for_0 = 1.0 / counts[0]
weight_for_1 = 1.0 / counts[1]


Number of positive samples in training data: 2527 (81.41% of total)


Les données étant déjà normalisées entre 0 et 1, on peut passer à la confection du modèle. On commence par un modèle avec des layers Dense basiques, on testera plus tard un modèle convolutionnel.

In [20]:
## CREATION DU MODELE

model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(x_train.shape[-1],)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()

#Ajout de layers convolutionnels, on part sur cette première approche

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 64)                39360     
_________________________________________________________________
dense_5 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 65        
Total params: 47,745
Trainable params: 47,745
Non-trainable params: 0
__________________________________________________

On définit ensuite les métriques utilisées pour calculer l'erreur du modèle. Dans les faits, seul le taux de faux positifs ou $FPR$ défini par $\frac{f_p}{f_p+t_n}$ nous intéresse ici. Comme celle-ci n'est pas déjà implémentée, on mesurera à la place la spécificité définie par $1 - FPR$.

In [49]:


metrix = [
    metrics.FalseNegatives(name="fn"),
    metrics.FalsePositives(name="fp"),
    metrics.TrueNegatives(name="tn"),
    metrics.TruePositives(name="tp"),
#     keras.metrics.Precision(name="precision"),
#     keras.metrics.Recall(name="recall"),
#     metrics.SpecificityAtSensitivity(0.5, name="spec")
    metrics.AUC(name="auc")
]

In [50]:
model.compile(
    optimizer=optimizers.Adam(1e-2), loss="binary_crossentropy", metrics=metrix
)

In [51]:
class_weight = {0: weight_for_0, 1: weight_for_1}

model.fit(
    x_train,
    y_train,
    batch_size=2048,
    epochs=30,
    verbose=2,
    validation_data=(x_val, y_val),
    class_weight=class_weight,
)

Epoch 1/30
2/2 - 2s - loss: 4.8539e-04 - fn: 991.0000 - fp: 66.0000 - tn: 511.0000 - tp: 1536.0000 - auc: 0.7434 - val_loss: 1.5484 - val_fn: 547.0000 - val_fp: 2.0000 - val_tn: 127.0000 - val_tp: 100.0000 - val_auc: 0.8374
Epoch 2/30
2/2 - 0s - loss: 4.8962e-04 - fn: 1360.0000 - fp: 60.0000 - tn: 517.0000 - tp: 1167.0000 - auc: 0.7663 - val_loss: 0.3335 - val_fn: 11.0000 - val_fp: 104.0000 - val_tn: 25.0000 - val_tp: 636.0000 - val_auc: 0.8930
Epoch 3/30
2/2 - 0s - loss: 4.8528e-04 - fn: 35.0000 - fp: 385.0000 - tn: 192.0000 - tp: 2492.0000 - auc: 0.8736 - val_loss: 0.3338 - val_fn: 51.0000 - val_fp: 25.0000 - val_tn: 104.0000 - val_tp: 596.0000 - val_auc: 0.9042
Epoch 4/30
2/2 - 0s - loss: 2.6091e-04 - fn: 467.0000 - fp: 77.0000 - tn: 500.0000 - tp: 2060.0000 - auc: 0.8972 - val_loss: 0.8246 - val_fn: 295.0000 - val_fp: 7.0000 - val_tn: 122.0000 - val_tp: 352.0000 - val_auc: 0.8944
Epoch 5/30
2/2 - 0s - loss: 3.4254e-04 - fn: 1003.0000 - fp: 39.0000 - tn: 538.0000 - tp: 1524.0000 - a